# Practice IV - Sentiment Analysis
## **1ra Parte Corpus preprocessing David**

In [1]:
import re, unicodedata, pandas as pd, os, spacy
from spacy.lang.es.stop_words import STOP_WORDS as SPACY_STOP_ES

os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"

Cargar Rest_Mex_2022

In [2]:
# Cargar corpus .xlsx
df = pd.read_excel('Recursos Profe/Rest_Mex_2022.xlsx', engine='openpyxl')
df.head(10)

,Title,Opinion,Polarity,Attraction
0,Pésimo lugar,"Piensen dos veces antes de ir a este hotel, te...",1,Hotel
1,No vayas a lugar de Eddie,Cuatro de nosotros fuimos recientemente a Eddi...,1,Restaurant
2,Mala relación calidad-precio,seguiré corta y simple: limpieza\n- bad. Tengo...,1,Hotel
3,Minusválido? ¡No te alojes aquí!,Al reservar un hotel con multipropiedad Mayan ...,1,Hotel
4,Es una porqueria no pierdan su tiempo,"No pierdan su tiempo ni dinero, venimos porque...",1,Hotel
5,El peor huevos Benedict jamás,Hoy teníamos el desayuno por segunda vez en po...,1,Restaurant
6,Definitivamente no volvería a hospedarme en Sa...,"El hotel en si no es malo, pero mi experiencia...",1,Hotel
7,Terrible,No estoy seguro de por qué este restaurante ti...,1,Restaurant
8,"Bebidas ADULTERADAS, FATAL mi experiencia!!!",Llegué a este hotel “por desgracia” mi reserva...,1,Hotel
9,Hotel ha ido cuesta abajo,Hemos estado viniendo a Villa la Estancia dura...,1,Hotel


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30212 entries, 0 to 30211
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Title       30210 non-null  object
 1   Opinion     30210 non-null  object
 2   Polarity    30212 non-null  int64 
 3   Attraction  30212 non-null  object
dtypes: int64(1), object(3)
memory usage: 944.3+ KB


In [4]:
df.shape

(30212, 4)

Concatenar Title + Opinion como texto de entrada.

In [5]:
# Convertir las columnas 'Title' y 'Opinion' a tipo string.
df['Title'] = df['Title'].astype(str)
df['Opinion'] = df['Opinion'].astype(str)

# Concatenar Title + Opinion como texto de entrada.
df['text'] = df['Title'] + ' ' + df['Opinion']

Dejar Polarity como etiqueta.

In [6]:
df['target'] = df['Polarity']

Normalización base:
- Minúsculas, quitar signos raros, espacios extra, etc.
- Manejo básico de stopwords.
- Manejo de negaciones.
- Manejo de repeticiones.
- Corrección básica de errores comunes.

Se añade:
- Emojis lexicon
- SEL_full

In [7]:
# Crear diccionario de traducción inglés -> español para términos comunes
en2es = {
    "front desk": "recepcion",
    "check-in": "check_in",
    "check in": "check_in",
    "early check-in": "check_in_temprano",
    "late check-out": "check_out_tarde",
    "check-out": "check_out",
    "check out": "check_out",
    "housekeeping": "limpieza_cuarto",
    "room service": "servicio_cuarto",
    "customer service": "servicio_cliente",
    "concierge": "conserje",
    "bellboy": "botones",
    "valet parking": "valet_parking",
    "self parking": "estacionamiento_autoservicio",
    "parking lot": "estacionamiento",
    "booking": "reserva",
    "overbooking": "sobre_reserva",
    "reservation": "reserva",
    "upgrade": "mejora_habitacion",
    "downgrade": "degradacion_habitacion",
    "no-show": "no_show",
    "walk-in": "llegada_sin_reserva",
    "timeshare": "tiempos_compartidos",
    "all inclusive": "todo_incluido",
    "half board": "media_pension",
    "full board": "pension_completa",
    "resort fee": "cargo_resort",
    "resort": "resort",
    "security deposit": "deposito_garantia",
    "refund": "reembolso",
    "chargeback": "contracargo",

    # --- Habitación / instalaciones ---
    "king bed": "cama_king",
    "queen bed": "cama_queen",
    "twin beds": "camas_individuales",
    "double bed": "cama_matrimonial",
    "sofa bed": "sofa_cama",
    "bunk bed": "literas",
    "extra bed": "cama_extra",
    "rollaway bed": "cama_plegable",
    "crib": "cuna",
    "mattress": "colchon",
    "pillow": "almohada",
    "blanket": "cobija",
    "linens": "sabanas",
    "towels": "toallas",
    "toilet paper": "papel_higienico",
    "bathroom": "bano",
    "bathtub": "banera",
    "hot tub": "jacuzzi",
    "jacuzzi": "jacuzzi",
    "shower": "ducha",
    "rain shower": "ducha_lluvia",
    "water pressure": "presion_agua",
    "air conditioning": "aire_acondicionado",
    "heater": "calefaccion",
    "safe box": "caja_fuerte",
    "mini bar": "minibar",
    "coffee maker": "cafetera",
    "hair dryer": "secadora_cabello",
    "iron": "plancha",
    "iron board": "tabla_planchar",
    "balcony": "balcon",
    "terrace": "terraza",
    "ocean view": "vista_mar",
    "sea view": "vista_mar",
    "garden view": "vista_jardin",
    "city view": "vista_ciudad",
    "soundproof": "insonorizado",
    "elevator": "elevador",
    "lift": "elevador",
    "wheelchair": "silla_ruedas",
    "wheelchair accessible": "acceso_silla_ruedas",
    "roll-in shower": "ducha_accesible",
    "accessible room": "habitacion_accesible",

    # --- Limpieza / plagas / olores ---
    "mold": "moho",
    "mildew": "moho",
    "leak": "fuga",
    "leaking": "fuga",
    "stain": "mancha",
    "stains": "manchas",
    "dusty": "polvoso",
    "dirty": "sucio",
    "smelly": "mal_olor",
    "bad smell": "mal_olor",
    "odor": "olor",
    "bedbug": "chinche_cama",
    "bedbugs": "chinches_cama",
    "cockroach": "cucaracha",
    "cockroaches": "cucarachas",
    "ants": "hormigas",
    "bugs": "insectos",

    # --- Personal / trato ---
    "staff": "personal",
    "friendly": "amable",
    "helpful": "servicial",
    "rude": "grosero",
    "impolite": "descortes",
    "unprofessional": "poco_profesional",
    "manager": "gerente",
    "host": "anfitrion",
    "hostess": "anfitriona",
    "waiter": "mesero",
    "waitress": "mesera",
    "server": "mesero",
    "bartender": "barman",
    "lifeguard": "salvavidas",

    # --- Comida / bebida ---
    "breakfast": "desayuno",
    "buffet": "buffet",
    "brunch": "brunch",
    "lunch": "comida",
    "dinner": "cena",
    "dessert": "postre",
    "appetizer": "entrada",
    "main course": "plato_fuerte",
    "menu": "menu",
    "dish": "platillo",
    "portion": "porcion",
    "taste": "sabor",
    "tasteless": "insipido",
    "delicious": "delicioso",
    "undercooked": "crudo",
    "overcooked": "sobre_cocido",
    "cold food": "comida_fria",
    "hot food": "comida_caliente",
    "drink": "bebida",
    "drinks": "bebidas",
    "beverage": "bebida",
    "beer": "cerveza",
    "wine": "vino",
    "cocktail": "coctel",
    "water": "agua",
    "ice": "hielo",
    "inclusive drinks": "bebidas_incluidas",

    # --- Zonas / amenidades ---
    "pool": "alberca",
    "infinity pool": "alberca_infinita",
    "pool bar": "bar_alberca",
    "beach": "playa",
    "beach towel": "toalla_playa",
    "private beach": "playa_privada",
    "spa": "spa",
    "sauna": "sauna",
    "steam room": "vapor",
    "gym": "gimnasio",
    "kids club": "club_ninos",
    "playground": "area_juegos",
    "shuttle": "transporte",
    "airport shuttle": "transporte_aeropuerto",
    "uber": "uber",
    "taxi": "taxi",
    "tour": "tour",
    "excursion": "excursion",

    # --- Tecnología / internet ---
    "wifi": "wifi",
    "wi-fi": "wifi",
    "internet": "internet",
    "signal": "senal",
    "coverage": "cobertura",
    "password": "contrasena",
    "tv": "tv",
    "smart tv": "smart_tv",
    "channel": "canal",
    "channels": "canales",
    "remote control": "control_remoto",
    "outlet": "contacto",
    "power outlet": "contacto",
    "charger": "cargador",

    # --- Precios / pagos / cargos ---
    "price": "precio",
    "expensive": "caro",
    "cheap": "barato",
    "fee": "cargo",
    "charge": "cargo",
    "extra charge": "cargo_extra",
    "hidden fee": "cargo_oculto",
    "tip": "propina",
    "tips": "propinas",

    # --- Ruido / seguridad ---
    "noise": "ruido",
    "noisy": "ruidoso",
    "thin walls": "paredes_delgadas",
    "security": "seguridad",
    "safe": "seguro",
    "unsafe": "inseguro",
    "theft": "robo",
    "stolen": "robado",

    # --- Valoraciones rápidas ---
    "amazing": "increible",
    "awesome": "increible",
    "great": "genial",
    "good": "bueno",
    "ok": "regular",
    "average": "promedio",
    "bad": "malo",
    "terrible": "terrible",
    "awful": "horrible",
    "horrible": "horrible",
    "disappointing": "decepcionante",
    "worth it": "vale_la_pena",
    "not worth it": "no_vale_la_pena",
}

In [8]:
stop_es = set(SPACY_STOP_ES)
negaciones = {"no","nunca","jamás","jamas","ni","sin", "tampoco","nadie","nada",
              "ningun","ningún","ninguno","ninguna","ningunos","ningunas"}
NEG_SINGLE = {"no","nunca","jamas","jamás","tampoco","nadie","nada","ni","sin",
              "ningun","ningún","ninguno","ninguna","ningunos","ningunas"}
NEG_BIGRAMS = {("ya","no"),("ni","siquiera"),("para","nada"),("en","absoluto"),("sin","que")}
stop_es_base = stop_es - negaciones
stop_en_basic = {"the","a","an","and","or","but","is","are","was","were","be","been","to","of",
                 "in","on","at","for","with","by","about","from","it","this","that","very",
                 "really","so","just","only","my","our","your","their","we","they"}
TIME_UNITS = {"am","pm","hora","horas","minuto","minutos","segundo","segundos"}

_re_url   = re.compile(r"https?://\S+|www\.\S+", re.I)
_re_email = re.compile(r"\b[\w.+-]+@[\w-]+\.[\w.-]+\b")
_re_money = re.compile(r"(?<!\d)(\$)\s*\d+[.,]?\d*")
_re_num = re.compile(r"(?<![\w%$])\d+[.,]?\d*(?![\w%$])")
_re_space = re.compile(r"\s+")
_re_risa  = re.compile(r"j+a+(j+a+)+", re.I)
_re_repe  = re.compile(r"(.)\1{2,}", re.UNICODE)
_re_emoji = re.compile("[" "\U0001F300-\U0001FAFF" "\U00002700-\U000027BF" "\U00002600-\U000026FF" "]+")
_re_punct = re.compile(r"[^\w\s%$]")

# Carga spaCy en español, excluyendo lo que no se necesita
nlp = spacy.load("es_core_news_sm", exclude=["ner","parser","senter","textcat","tok2vec"])
BATCH, NPROC = 4000, 6

# Emoji Lexicon
emo_df  = pd.read_excel("Recursos Profe/Emojis lexicon.XLSX")
emo_df.columns = [str(c).strip().lower() for c in emo_df.columns]
emo_cols = ["anger","anticipation","disgust","fear","joy","sadness","surprise","trust"]
sent_cols = ["negative","positive"]
emoji_list = emo_df["emoji"].astype(str).tolist()
weights_emotions = {
    e: emo_df.set_index("emoji")[e].to_dict() for e in emo_cols
}
weights_sent = {
    s: emo_df.set_index("emoji")[s].to_dict() for s in sent_cols
}

def emoji_weighted_features(raw_text: str):
    # Cuenta ocurrencias por emoji (algunos pueden repetirse)
    # Nota: .count() funciona bien para emojis de un caracter; si tuvieras secuencias, usa regex.
    counts = {em: raw_text.count(em) for em in emoji_list}
    total = sum(counts.values())

    # Acumula puntajes ponderados por ocurrencia
    feat = {}
    # a) emociones básicas
    for e in emo_cols:
        s = 0.0
        wdict = weights_emotions[e]
        for em, c in counts.items():
            if c:
                w = wdict.get(em, 0.0)
                s += c * float(w)
        feat[f"emoji_{e}_score"] = s

    # b) polaridad agregada
    pos = neg = 0.0
    for em, c in counts.items():
        if c:
            pos += c * float(weights_sent["positive"].get(em, 0.0))
            neg += c * float(weights_sent["negative"].get(em, 0.0))
    feat["emoji_pos_score"] = pos
    feat["emoji_neg_score"] = neg
    feat["emoji_score"] = pos - neg

    # c) conteos crudos y densidades
    feat["emoji_count"] = int(total)
    # normaliza por longitud para que no “premie” textos largos (opcional)
    L = max(len(raw_text), 1)
    feat["emoji_density"] = total / L

    return pd.Series(feat)

# Lemtacización
def rejoin_negations_after_lemma(text_lemmatized: str):
    toks = text_lemmatized.split()
    out = []; i = 0
    while i < len(toks):
        tk = toks[i]
        # Bigramas
        if i+1 < len(toks) and (tk, toks[i+1]) in NEG_BIGRAMS:
            pair = f"{tk}_{toks[i+1]}"
            if i+2 < len(toks) and toks[i+2] not in stop_es_base:
                out.append(f"{pair}_{toks[i+2]}"); i += 3
            else:
                out.append(pair); i += 2
            continue
        # 'sin' solo
        if tk == "sin":
            if i+1 < len(toks) and toks[i+1] not in stop_es_base:
                out.append(f"sin_{toks[i+1]}"); i += 2
            else:
                out.append("sin"); i += 1
            continue
        # Un solo token (no, nunca, ni, tampoco, ningún…)
        if tk in NEG_SINGLE:
            if i+1 < len(toks):
                nxt = toks[i+1]
                if (nxt not in stop_es_base and nxt != "num" and nxt not in TIME_UNITS):
                    out.append(f"{tk}_{nxt}"); i += 2; continue
            out.append(tk); i += 1; continue
        out.append(tk); i += 1
    return " ".join(out)

# Funciones de normalización
def strip_accents(s): 
    nfkd = unicodedata.normalize("NFKD", s)
    return "".join(ch for ch in nfkd if not unicodedata.combining(ch))

def reduce_repeated(text): return _re_repe.sub(r"\1\1", text)

def replace_en_domain_words(text):
    for en, es in en2es.items():
        text = re.sub(rf"\b{re.escape(en)}\b", es, text, flags=re.I)
    return text

def normalize_text(
    s: str,
    *,
    use_en2es: bool = True,
    remove_stopwords: bool = True
) -> str:
    s = str(s).lower().replace("’","'").replace("“",'"').replace("”",'"')
    s = _re_url.sub(" url ", s); s = _re_email.sub(" email ", s)
    s = s.replace(":)", " emoticon_positivo ").replace("(:", " emoticon_positivo ")
    s = s.replace(":(", " emoticon_negativo ").replace("):", " emoticon_negativo ")
    # Nota: para el texto normalizado quitamos los emojis (se calculan features aparte sobre el texto crudo)
    s = _re_emoji.sub(" emoji ", s)
    s = _re_risa.sub(" jaja ", s); s = reduce_repeated(s)
    s = _re_money.sub(" dinero ", s); s = _re_num.sub(" num ", s)
    s = _re_punct.sub(" ", s)
    if use_en2es:
        s = replace_en_domain_words(s)
    s = _re_space.sub(" ", s).strip()
    if remove_stopwords:
        toks = [t for t in s.split() if (t not in stop_es_base and t not in stop_en_basic)]
        s = " ".join(toks)
    return s

def maybe_lemmatize(texts, do_lemma: bool = True):
    if not do_lemma:
        # Si no lematizamos, devolvemos tal cual
        return [t for t in texts]
    lemmas_local = []
    for doc in nlp.pipe(texts, batch_size=BATCH, n_process=NPROC):
        lemmas_local.append(" ".join(t.lemma_ for t in doc))
    return lemmas_local

def maybe_strip_accents(texts, do_strip: bool = True):
    if not do_strip:
        return texts
    return [strip_accents(t) for t in texts]

def maybe_join_negations(texts, do_join: bool = True):
    if not do_join:
        return texts
    return [rejoin_negations_after_lemma(t) for t in texts]

def add_emoji_features(df_out, do_emoji_feats: bool = True):
    if not do_emoji_feats:
        return df_out
    feats = df_out["text"].apply(emoji_weighted_features)
    df_out = pd.concat([df_out, feats], axis=1)
    return df_out

def add_sel_features(df_out, do_sel_feats: bool = True):
    if not do_sel_feats:
        return df_out
    sel_feats = df_out["text_norm"].apply(sel_features)
    df_out = pd.concat([df_out, sel_feats.rename(columns={"sel_score":"sel_score"})], axis=1)
    return df_out

# SEL
sel = pd.read_csv("Recursos Profe/SEL_full.txt", sep=r"\s*\t\s*", engine="python")
sel["key"] = sel["Palabra"].str.lower().apply(strip_accents)
pfa = dict(zip(sel["key"], sel["PFA"]))
cat = dict(zip(sel["key"], sel["Categoría"].str.lower()))
emociones = sorted(sel["Categoría"].str.lower().unique())

def sel_features(text_norm):
    toks = text_norm.split()
    score = 0.0
    counts = {e: 0 for e in emociones}
    for t in toks:
        w = t.replace("no_",""); inv = t.startswith("no_")
        if w in pfa: score += (-pfa[w] if inv else pfa[w])
        if w in cat: counts[cat[w]] += (-1 if inv else 1)
    counts["sel_score"] = score
    return pd.Series(counts)

Generar multiples archivos con el pre procesamiento.

In [ ]:
# Se define multiples formas de normalizar el texto y agregar features.
VARIANTS = [
    # Todo activado
    dict(name="all_on",
         strip_accents=True, en2es=True, stopwords=True, lemma=True, neg_join=True, emoji_feats=True, sel_feats=True),

    # Uno a uno desactivados
    dict(name="no_accents",
         strip_accents=False, en2es=True, stopwords=True, lemma=True, neg_join=True, emoji_feats=True, sel_feats=True),
    dict(name="no_en2es",
         strip_accents=True, en2es=False, stopwords=True, lemma=True, neg_join=True, emoji_feats=True, sel_feats=True),
    dict(name="no_stopwords",
         strip_accents=True, en2es=True, stopwords=False, lemma=True, neg_join=True, emoji_feats=True, sel_feats=True),
    dict(name="no_lemma",
         strip_accents=True, en2es=True, stopwords=True, lemma=False, neg_join=True, emoji_feats=True, sel_feats=True),
    dict(name="no_negjoin",
         strip_accents=True, en2es=True, stopwords=True, lemma=True, neg_join=False, emoji_feats=True, sel_feats=True),
    dict(name="no_emoji_feats",
         strip_accents=True, en2es=True, stopwords=True, lemma=True, neg_join=True, emoji_feats=False, sel_feats=True),
    dict(name="no_sel",
         strip_accents=True, en2es=True, stopwords=True, lemma=True, neg_join=True, emoji_feats=True, sel_feats=False),

    # Limpieza ligera sin extras
    dict(name="minimal",
         strip_accents=False, en2es=False, stopwords=False, lemma=False, neg_join=False, emoji_feats=False, sel_feats=False),
]

In [10]:
# Se define donde se guardaran los archivos procesados
BASE_OUT = "Archivos_Procesados/Rest_Mex_2022_preprocessed__"

for cfg in VARIANTS:
    print(f"Generando variante: {cfg['name']} -> {cfg}")
    df_var = df.copy()

    # Normalización con switches (en2es + stopwords)
    df_var["text_clean_tmp"] = df_var["text"].apply(
        lambda s: normalize_text(s, use_en2es=cfg["en2es"], remove_stopwords=cfg["stopwords"])
    )

    # Lematización opcional
    lemmas_local = maybe_lemmatize(df_var["text_clean_tmp"].tolist(), do_lemma=cfg["lemma"])

    # Acentos opcionales
    lemmas_local = maybe_strip_accents(lemmas_local, do_strip=cfg["strip_accents"])

    # Unión de negaciones opcional
    lemmas_local = maybe_join_negations(lemmas_local, do_join=cfg["neg_join"])

    df_var["text_norm"] = lemmas_local
    df_var.drop(columns=["text_clean_tmp"], inplace=True)

    # Features opcionales emojis / SEL
    df_var = add_emoji_features(df_var, do_emoji_feats=cfg["emoji_feats"])
    df_var = add_sel_features(df_var, do_sel_feats=cfg["sel_feats"])

    # Guarda archivo
    out_file = f"{BASE_OUT}{cfg['name']}.xlsx"
    df_var.to_excel(out_file, index=False, engine="openpyxl")

Generando variante: all_on -> {'name': 'all_on', 'strip_accents': True, 'en2es': True, 'stopwords': True, 'lemma': True, 'neg_join': True, 'emoji_feats': True, 'sel_feats': True}
Generando variante: no_accents -> {'name': 'no_accents', 'strip_accents': False, 'en2es': True, 'stopwords': True, 'lemma': True, 'neg_join': True, 'emoji_feats': True, 'sel_feats': True}
Generando variante: no_en2es -> {'name': 'no_en2es', 'strip_accents': True, 'en2es': False, 'stopwords': True, 'lemma': True, 'neg_join': True, 'emoji_feats': True, 'sel_feats': True}
Generando variante: no_stopwords -> {'name': 'no_stopwords', 'strip_accents': True, 'en2es': True, 'stopwords': False, 'lemma': True, 'neg_join': True, 'emoji_feats': True, 'sel_feats': True}
Generando variante: no_lemma -> {'name': 'no_lemma', 'strip_accents': True, 'en2es': True, 'stopwords': True, 'lemma': False, 'neg_join': True, 'emoji_feats': True, 'sel_feats': True}
Generando variante: no_negjoin -> {'name': 'no_negjoin', 'strip_accents':

# **Parte 2 Alberto**

**Configurar columnas y asegurar split 80/20**

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy as np
from scipy.sparse import hstack, csr_matrix

In [16]:
print(df.columns)

# Ajustar nombres según lo que ya creaste
TEXT_COL = 'text'      # columna con Title + Opinion concatenado
LABEL_COL = 'target'   # o 'Polarity' 

display(df[[TEXT_COL, LABEL_COL]].head())


Index(['Title', 'Opinion', 'Polarity', 'Attraction', 'text', 'target',
       'text_norm', 'emoji_anger_score', 'emoji_anticipation_score',
       'emoji_disgust_score', 'emoji_fear_score', 'emoji_joy_score',
       'emoji_sadness_score', 'emoji_surprise_score', 'emoji_trust_score',
       'emoji_pos_score', 'emoji_neg_score', 'emoji_score', 'emoji_count',
       'emoji_density', 'sel_score', 'alegría', 'enojo', 'miedo', 'repulsión',
       'sorpresa', 'tristeza'],
      dtype='object')


,text,target
0,Pésimo lugar Piensen dos veces antes de ir a e...,1
1,No vayas a lugar de Eddie Cuatro de nosotros f...,1
2,Mala relación calidad-precio seguiré corta y s...,1
3,Minusválido? ¡No te alojes aquí! Al reservar u...,1
4,Es una porqueria no pierdan su tiempo No pierd...,1


**Split 80/20 con shuffle=True y random_state=0**

In [17]:
X_train_text, X_test_text, y_train, y_test = train_test_split(
    df[TEXT_COL],
    df[LABEL_COL],
    test_size=0.2,
    shuffle=True,
    random_state=0,
    stratify=df[LABEL_COL]  #por el desbalance
)

print("Tamaño train:", len(X_train_text))
print("Tamaño test:", len(X_test_text))

Tamaño train: 24169
Tamaño test: 6043


**Funciones de vectorización (binaria, frecuencia, TF-IDF)**  
Estas funciones:

Transforman texto → matriz numérica.

Mantienen el mismo split porque el vectorizador se ajusta SOLO con X_train_text y luego se aplica a X_test_text.

In [18]:
def vectorize_binary(X_train_text, X_test_text, **kwargs):
    """
    Representación binarizada: 1 si la palabra aparece, 0 si no.
    """
    vect_bin = CountVectorizer(binary=True, **kwargs)
    X_train_bin = vect_bin.fit_transform(X_train_text)
    X_test_bin = vect_bin.transform(X_test_text)
    return X_train_bin, X_test_bin, vect_bin

def vectorize_frequency(X_train_text, X_test_text, **kwargs):
    """
    Representación por frecuencia: conteo de ocurrencias de cada término.
    """
    vect_freq = CountVectorizer(binary=False, **kwargs)
    X_train_freq = vect_freq.fit_transform(X_train_text)
    X_test_freq = vect_freq.transform(X_test_text)
    return X_train_freq, X_test_freq, vect_freq

def vectorize_tfidf(X_train_text, X_test_text, **kwargs):
    """
    Representación TF-IDF: pondera términos según importancia en documentos.
    """
    vect_tfidf = TfidfVectorizer(**kwargs)
    X_train_tfidf = vect_tfidf.fit_transform(X_train_text)
    X_test_tfidf = vect_tfidf.transform(X_test_text)
    return X_train_tfidf, X_test_tfidf, vect_tfidf

**Ejemplo rápido para generar las tres:**

In [19]:
X_train_bin, X_test_bin, vect_bin = vectorize_binary(X_train_text, X_test_text)
X_train_freq, X_test_freq, vect_freq = vectorize_frequency(X_train_text, X_test_text)
X_train_tfidf, X_test_tfidf, vect_tfidf = vectorize_tfidf(X_train_text, X_test_text)

print("Binaria:", X_train_bin.shape, X_test_bin.shape)
print("Frecuencia:", X_train_freq.shape, X_test_freq.shape)
print("TF-IDF:", X_train_tfidf.shape, X_test_tfidf.shape)

Binaria: (24169, 45691) (6043, 45691)
Frecuencia: (24169, 45691) (6043, 45691)
TF-IDF: (24169, 45691) (6043, 45691)


**Léxicos de sentimiento (features extra)**  
Aquí agregamos conteo de palabras positivas y negativas por documento.

In [20]:
# Ejemplo simple de léxicos (AMPLÍA ESTO para la práctica real)
positive_lexicon = {
    "bueno","excelente","perfecto","maravilloso","increíble","bonito","agradable",
    "recomendado","limpio","rápido","delicioso","rico","amable","fantástico","genial"
}

negative_lexicon = {
    "malo","terrible","horrible","pésimo","lento","sucio","caro",
    "decepcionante","desagradable","asqueroso","frío","tardado","estresante","feo"
}

def count_sentiment_words(text, pos_lex=positive_lexicon, neg_lex=negative_lexicon):
    tokens = text.split()
    pos_count = sum(1 for t in tokens if t in pos_lex)
    neg_count = sum(1 for t in tokens if t in neg_lex)
    return pos_count, neg_count

def build_lexicon_features(text_series):
    """
    Regresa una matriz numpy de shape (n_docs, 2):
    [conteo_pos, conteo_neg]
    """
    feats = [count_sentiment_words(str(t)) for t in text_series]
    return np.array(feats)

**prueba**

In [21]:
lex_train = build_lexicon_features(X_train_text)
lex_test = build_lexicon_features(X_test_text)

print("Léxicos train:", lex_train.shape)
print("Léxicos test:", lex_test.shape)
lex_train[:5]


Léxicos train: (24169, 2)
Léxicos test: (6043, 2)


array([[0, 0],
       [0, 0],
       [1, 0],
       [0, 0],
       [5, 1]])

**Emojis / Emoticonos (features extra)**  
Features típicos:

Conteo de emojis/emoticonos positivos.

Conteo de emojis/emoticonos negativos.

(Opcional) Conteo de signos de exclamación para intensidad.

In [22]:
emoji_pattern = re.compile(
    r'['
    r'\U0001F300-\U0001F5FF'  # símbolos y pictogramas
    r'\U0001F600-\U0001F64F'  # emoticonos
    r'\U0001F680-\U0001F6FF'  # transporte y mapas
    r'\U0001F700-\U0001F77F'
    r'\U0001F780-\U0001F7FF'
    r'\U0001F800-\U0001F8FF'
    r'\U0001F900-\U0001F9FF'
    r'\U0001FA00-\U0001FAFF'
    r'\u2600-\u26FF'          # símbolos varios
    r'\u2700-\u27BF'
    r']'
)

def extract_emojis(text):
    if not isinstance(text, str):
        return []
    return emoji_pattern.findall(text)

# Definir positivo / negativo 
# 1 = muy negativo, 2 = negativo, 3 = neutro, 4 = positivo, 5 = muy positivo
POS_LABELS = {4, 5}
NEG_LABELS = {1, 2}

pos_counts = {}
neg_counts = {}

for txt, label in zip(df[TEXT_COL], df[LABEL_COL]):
    emojis = extract_emojis(txt)
    if not emojis:
        continue

    # normalizamos label como string
    lab = str(label).strip()

    # conjuntos ya como strings para comparar
    pos_set = {str(l) for l in POS_LABELS}
    neg_set = {str(l) for l in NEG_LABELS}

    if lab in pos_set:
        for e in emojis:
            pos_counts[e] = pos_counts.get(e, 0) + 1
    elif lab in neg_set:
        for e in emojis:
            neg_counts[e] = neg_counts.get(e, 0) + 1

# Emojis clasificados según en qué clase aparecen más
positive_emojis = {e for e in pos_counts if pos_counts[e] > neg_counts.get(e, 0)}
negative_emojis = {e for e in neg_counts if neg_counts[e] > pos_counts.get(e, 0)}

print("Emojis positivos detectados:", positive_emojis)
print("Emojis negativos detectados:", negative_emojis)

# Emoticonos comunes 
positive_emoticons = {":)", ":-)", ":D", "(:", "=)", ";)", ";-)", ":')"}
negative_emoticons = {":(", ":-(", "):", "='(", ":'(", "D:", ">:(", ":-/"}

# Funciones de features
def emoji_emoticon_features(text):
    t = str(text)

    # conteo de emojis según los sets aprendidos
    pos = sum(ch in positive_emojis for ch in t)
    neg = sum(ch in negative_emojis for ch in t)

    # conteo de emoticonos
    for emo in positive_emoticons:
        pos += t.count(emo)
    for emo in negative_emoticons:
        neg += t.count(emo)

    # intensidad con signos de exclamación
    exclam = t.count("!")

    return pos, neg, exclam

def build_emoji_features(text_series):
    """
    Regresa matriz (n_docs, 3):
    [emoji_pos+emo_pos, emoji_neg+emo_neg, exclamaciones]
    """
    feats = [emoji_emoticon_features(t) for t in text_series]
    return np.array(feats)

# Generar features para train/test
emoji_train = build_emoji_features(X_train_text)
emoji_test = build_emoji_features(X_test_text)

print("Emoji feats train:", emoji_train.shape)
print("Emoji feats test:", emoji_test.shape)
emoji_train[:5]

Emojis positivos detectados: {'🎸', '💤', '🌹', '🏊', '🙌', '👦', '😜', '🐙', '🤩', '😬', '🤘', '🎼', '🪂', '🐲', '♡', '😀', '☺', '🏄', '😍', '🍭', '🦪', '🎉', '🥂', '✌', '🌞', '💧', '🤍', '👧', '♀', '🏿', '🏻', '🍺', '🤪', '🍾', '🎂', '🍻', '🌭', '☀', '🔟', '👏', '😊', '🎇', '🥗', '👩', '🍸', '🤣', '😄', '👯', '🌴', '🥰', '🎶', '😃', '🥤', '👌', '🎥', '😚', '🦎', '👐', '🔥', '🤙', '🌻', '😌', '🏖', '🎈', '🥺', '✔', '🙂', '👍', '✋', '🌅', '☹', '🍟', '🎭', '👊', '🦐', '🌮', '😆', '🖤', '🌈', '🍲', '🤤', '🏡', '💆', '♧', '♂', '💙', '✅', '🏩', '🙏', '👪', '💓', '😉', '✨', '🏅', '👓', '🔙', '🤽', '🍔', '🐷', '🎤', '😋', '😸', '🌎', '♥', '😪', '🍯', '😇', '😈', '💐', '❤', '🙈', '🕶', '💁', '🎊', '😭', '🍕', '🍹', '🏼', '⛱', '🙋', '💖', '🏽', '👅', '🏓', '🌊', '👙', '😑', '💎', '😂', '💕', '😝', '🍝', '🌷', '❄', '🌋', '⚽', '❗', '🏐', '🌚', '🙊', '💛', '🌺', '👀', '💪', '👨', '💗', '😁', '🍣', '🏾', '😷', '😖', '🤓', '😎', '🙃', '🏆', '🔆', '🤟', '😅', '🤠', '💯', '💜', '🤗', '👎', '😢', '🦩', '🥘', '🤔', '😛', '🦞', '🏝', '🍜', '😘', '😱', '😦', '👋', '🥳'}
Emojis negativos detectados: {'😡', '🙁', '😠', '😔', '🤚', '😓', '😳'}
Emoji feats train: (24169

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

**Combinar representaciones + features extra**

In [23]:
def add_extra_features(X_train_base, X_test_base, extra_train, extra_test):
    """
    Concatena la matriz dispersa base con features densos extra.
    """
    extra_train_sparse = csr_matrix(extra_train)
    extra_test_sparse = csr_matrix(extra_test)
    
    X_train_comb = hstack([X_train_base, extra_train_sparse])
    X_test_comb = hstack([X_test_base, extra_test_sparse])
    
    return X_train_comb, X_test_comb

# Ejemplo: TF-IDF + léxicos + emojis
extra_train = np.hstack([lex_train, emoji_train])
extra_test = np.hstack([lex_test, emoji_test])

X_train_tfidf_ext, X_test_tfidf_ext = add_extra_features(
    X_train_tfidf, X_test_tfidf,
    extra_train, extra_test
)

print("TF-IDF simple:", X_train_tfidf.shape)
print("TF-IDF + extras:", X_train_tfidf_ext.shape)

TF-IDF simple: (24169, 45691)
TF-IDF + extras: (24169, 45696)


# **Tabla/resumen**  
| Representación      | Descripción                                                                 | Ventajas                                                   | Desventajas                                                    |
|---------------------|-----------------------------------------------------------------------------|------------------------------------------------------------|----------------------------------------------------------------|
| Binarizada          | Cada término se representa con 0/1 según aparezca en el documento.         | Simple, reduce impacto de repeticiones, útil con muchos docs. | Pierde información de frecuencia; menos expresiva.            |
| Frecuencia (BoW)    | Cada término es el número de veces que aparece en el documento.            | Fácil de interpretar, buena base para muchos modelos.      | Favorece palabras muy frecuentes; no considera importancia global. |
| TF-IDF              | Pondera frecuencia local vs. frecuencia global del término.                | Resalta términos relevantes; muy usada en IR y NLP clásico.| Más compleja; puede sobreajustar si no se regula.              |
| Léxicos de sentimiento (extra) | Features numéricos con conteo de palabras positivas/negativas.          | Introduce conocimiento lingüístico explícito.             | Depende de la calidad/cobertura del léxico; sensible al dominio. |
| Emojis/Emoticonos (extra)      | Conteo de emojis/emoticonos positivos/negativos e intensidad (!).       | Captura señales afectivas típicas de redes/comentarios.   | No siempre aparecen; puede ser ruidoso si se usa solo.         |


In [ ]:
import pandas as pd
import numpy as np
import pickle
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, KFold, cross_validate, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from scipy.sparse import hstack, csr_matrix
import re
import os

# Para manejo de desbalanceo
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

# Cargar el dataset preprocesado
df = pd.read_excel('Rest_Mex_2022_preprocessed.xlsx')

# Configurar columnas
TEXT_COL = 'text_norm'
LABEL_COL = 'target'

print("Distribución de clases en el dataset completo:")
print(df[LABEL_COL].value_counts().sort_index())

# Split 80/20
X_train_text, X_test_text, y_train, y_test = train_test_split(
    df[TEXT_COL],
    df[LABEL_COL],
    test_size=0.2,
    shuffle=True,
    random_state=0,
    stratify=df[LABEL_COL]
)

print(f"\nTamaño train: {len(X_train_text)}")
print(f"Tamaño test: {len(X_test_text)}")
print("Distribución en train:", np.bincount(y_train))
print("Distribución en test:", np.bincount(y_test))

# =============================================================================
# DIFERENTES REPRESENTACIONES DE TEXTO
# =============================================================================

def create_vectorizers():
    """Crea diferentes representaciones vectoriales"""
    vectorizers = {
        'CountBinary': CountVectorizer(binary=True, max_features=5000),
        'CountFreq': CountVectorizer(binary=False, max_features=5000),
        'TFIDF': TfidfVectorizer(max_features=5000),
        'TFIDF_ngrams': TfidfVectorizer(ngram_range=(1, 2), max_features=5000)
    }
    return vectorizers

# =============================================================================
# CARACTERÍSTICAS LÉXICAS SEL
# =============================================================================

def load_sel():
    lexicon_sel = {}
    try:
        input_file = open('Recursos Profe/SEL_full.txt', 'r', encoding='utf-8')
        for line in input_file:
            palabras = line.split("\t")
            if len(palabras) >= 7:
                palabras[6] = re.sub('\n', '', palabras[6])
                pair = (palabras[6], palabras[5])
                if palabras[0] not in lexicon_sel:
                    lista = [pair]
                    lexicon_sel[palabras[0]] = lista
                else:
                    lexicon_sel[palabras[0]].append(pair)
        input_file.close()
        if 'Palabra' in lexicon_sel:
            del lexicon_sel['Palabra']
    except Exception as e:
        print(f"Error cargando SEL: {e}")
        lexicon_sel = {}
    return lexicon_sel

def getSELFeatures(cadenas, lexicon_sel):
    features = []
    for cadena in cadenas:
        valores = {emocion: 0.0 for emocion in ['alegria', 'tristeza', 'enojo', 'repulsion', 'miedo', 'sorpresa']}
        
        cadena_palabras = re.split('\s+', cadena)
        
        for palabra in cadena_palabras:
            if palabra in lexicon_sel:
                caracteristicas = lexicon_sel[palabra]
                for emocion, valor in caracteristicas:
                    emocion_key = emocion.lower().strip()
                    if emocion_key in valores:
                        valores[emocion_key] += float(valor)
        
        # Características adicionales
        valores['acumuladopositivo'] = valores['alegria'] + valores['sorpresa']
        valores['acumuladonegative'] = valores['enojo'] + valores['miedo'] + valores['repulsion'] + valores['tristeza']
        
        features.append(valores)
    
    return features

# Cargar léxico SEL
lexicon_sel = load_sel()
sel_cols = ['alegria', 'tristeza', 'enojo', 'repulsion', 'miedo', 'sorpresa', 'acumuladopositivo', 'acumuladonegative']

# =============================================================================
# ESTRATEGIAS DE BALANCEO
# =============================================================================

balance_strategies = {
    'none': None,
    'oversample': RandomOverSampler(random_state=0),
    'undersample': RandomUnderSampler(random_state=0),
    'smote': SMOTE(random_state=0)
}

# =============================================================================
# MODELOS E HIPERPARÁMETROS
# =============================================================================

models_config = {
    'LogisticRegression': {
        'model': LogisticRegression(max_iter=10000, random_state=0),
        'params': {
            'C': [0.1, 1, 10, 50, 100],  # Rango más amplio para regularización
            'solver': ['liblinear', 'lbfgs'],
            'penalty': ['l1', 'l2'] # Probar regularización L1 y L2
        }
    },
    'RandomForest': {
        'model': RandomForestClassifier(random_state=0),
        'params': {
            'n_estimators': [100, 200, 300], # Más árboles para mayor estabilidad
            'max_depth': [10, 20, 30, None], # Mayor profundidad
            'min_samples_split': [2, 5, 10], # Control de sobreajuste
            'min_samples_leaf': [1, 2, 4],   # Control de sobreajuste
            'class_weight': [None, 'balanced']
        }
    },
    'SVM': {
        'model': SVC(random_state=0),
        'params': {
            'C': [0.1, 1, 10, 100], # Rango más amplio para regularización
            'kernel': ['linear', 'rbf'],
            'gamma': ['scale', 'auto', 0.1, 1], # Clave para el kernel RBF
            'class_weight': [None, 'balanced']
        }
    },
    'NaiveBayes': {
        'model': MultinomialNB(),
        'params': {
            'alpha': [0.01, 0.1, 0.5, 1.0, 2.0] # Ajuste más fino del suavizado
        }
    }
}

# =============================================================================
# EXPERIMENTACIÓN CON VALIDACIÓN CRUZADA (5 FOLDS)
# =============================================================================

def evaluate_model_with_cv(model, X, y, balance_strategy=None):
    """Evalúa modelo con 5-fold CV y estrategia de balanceo"""
    kf = KFold(n_splits=5, shuffle=True, random_state=0)
    
    if balance_strategy:
        # Crear pipeline con balanceo
        pipeline = Pipeline([
            ('sampler', balance_strategy),
            ('classifier', model)
        ])
    else:
        pipeline = Pipeline([
            ('classifier', model)
        ])
    
    cv_results = cross_validate(
        pipeline, X, y, 
        cv=kf, 
        scoring='f1_macro',
        n_jobs=-1,
        return_train_score=True
    )
    
    return {
        'mean_train_f1': np.mean(cv_results['train_score']),
        'std_train_f1': np.std(cv_results['train_score']),
        'mean_test_f1': np.mean(cv_results['test_f1_macro']),
        'std_test_f1': np.std(cv_results['test_f1_macro'])
    }

# Almacenar resultados
results = []

print("=" * 80)
print("COMIENZO DE EXPERIMENTACIÓN CON 5-FOLD CV")
print("=" * 80)

# Probar diferentes combinaciones
vectorizers = create_vectorizers()
lexicon_sel = load_sel()

for vec_name, vectorizer in vectorizers.items():
    print(f"\n--- Probando representación: {vec_name} ---")
    
    # Vectorizar texto
    X_train_vec = vectorizer.fit_transform(X_train_text)
    X_test_vec = vectorizer.transform(X_test_text)
    
    # Obtener características SEL
    if lexicon_sel:
        sel_features_train = getSELFeatures(X_train_text, lexicon_sel)
        sel_features_test = getSELFeatures(X_test_text, lexicon_sel)
        sel_train_array = np.array([[f[col] for col in sel_cols] for f in sel_features_train])
        sel_test_array = np.array([[f[col] for col in sel_cols] for f in sel_features_test])
        
        # Combinar características
        X_train_combined = hstack([X_train_vec, csr_matrix(sel_train_array)])
        X_test_combined = hstack([X_test_vec, csr_matrix(sel_test_array)])
    else:
        X_train_combined = X_train_vec
        X_test_combined = X_test_vec
    
    for model_name, model_config in models_config.items():
        for balance_name, balance_strategy in balance_strategies.items():
            print(f"  Modelo: {model_name} | Balanceo: {balance_name}")
            
            try:
                # Evaluar con CV
                cv_results = evaluate_model_with_cv(
                    model_config['model'], 
                    X_train_combined, 
                    y_train, 
                    balance_strategy
                )
                
                # Guardar resultados
                result = {
                    'representation': vec_name,
                    'model': model_name,
                    'balancing': balance_name,
                    'mean_train_f1': cv_results['mean_train_f1'],
                    'std_train_f1': cv_results['std_train_f1'],
                    'mean_test_f1': cv_results['mean_test_f1'],
                    'std_test_f1': cv_results['std_test_f1']
                }
                results.append(result)
                
                print(f"    F1-macro: {cv_results['mean_test_f1']:.4f} ± {cv_results['std_test_f1']:.4f}")
                
            except Exception as e:
                print(f"    Error: {e}")
                continue

# =============================================================================
# ANÁLISIS DE RESULTADOS Y SELECCIÓN DEL MEJOR MODELO
# =============================================================================

# Crear DataFrame con resultados
results_df = pd.DataFrame(results)
print("\n" + "=" * 80)
print("RESULTADOS COMPLETOS DE EXPERIMENTACIÓN")
print("=" * 80)
print(results_df.round(4))

# Encontrar mejor combinación
best_result_idx = results_df['mean_test_f1'].idxmax()
best_result = results_df.loc[best_result_idx]

print("\n" + "=" * 80)
print("MEJOR COMBINACIÓN ENCONTRADA")
print("=" * 80)
print(f"Representación: {best_result['representation']}")
print(f"Modelo: {best_result['model']}")
print(f"Balanceo: {best_result['balancing']}")
print(f"F1-macro (CV): {best_result['mean_test_f1']:.4f} ± {best_result['std_test_f1']:.4f}")

# =============================================================================
# ENTRENAMIENTO FINAL DEL MEJOR MODELO
# =============================================================================

print("\n" + "=" * 80)
print("ENTRENAMIENTO FINAL DEL MEJOR MODELO")
print("=" * 80)

# Recrear la mejor representación
best_vectorizer = vectorizers[best_result['representation']]
X_train_vec = best_vectorizer.fit_transform(X_train_text)
X_test_vec = best_vectorizer.transform(X_test_text)

# Agregar características SEL si están disponibles
if lexicon_sel:
    sel_features_train = getSELFeatures(X_train_text, lexicon_sel)
    sel_features_test = getSELFeatures(X_test_text, lexicon_sel)
    sel_train_array = np.array([[f[col] for col in sel_cols] for f in sel_features_train])
    sel_test_array = np.array([[f[col] for col in sel_cols] for f in sel_features_test])
    X_train_final = hstack([X_train_vec, csr_matrix(sel_train_array)])
    X_test_final = hstack([X_test_vec, csr_matrix(sel_test_array)])
else:
    X_train_final = X_train_vec
    X_test_final = X_test_vec

# Configurar el mejor modelo con balanceo
best_model_config = models_config[best_result['model']]
best_balance_strategy = balance_strategies[best_result['balancing']]

if best_balance_strategy:
    final_model = Pipeline([
        ('sampler', best_balance_strategy),
        ('classifier', best_model_config['model'])
    ])
else:
    final_model = Pipeline([
        ('classifier', best_model_config['model'])
    ])

# Entrenar modelo final
final_model.fit(X_train_final, y_train)

# Predicciones finales
y_pred = final_model.predict(X_test_final)

# Métricas finales
test_accuracy = accuracy_score(y_test, y_pred)
test_f1_macro = f1_score(y_test, y_pred, average='macro')
test_f1_weighted = f1_score(y_test, y_pred, average='weighted')

print(f"\n=== RESULTADOS FINALES EN TEST ===")
print(f"Accuracy: {test_accuracy:.4f}")
print(f"F1-macro: {test_f1_macro:.4f}")
print(f"F1-weighted: {test_f1_weighted:.4f}")

# Matriz de confusión
print(f"\nMatriz de confusión:")
print(confusion_matrix(y_test, y_pred, labels=[1, 2, 3, 4, 5]))

# Reporte de clasificación detallado
target_names = ['1', '2', '3', '4', '5']
print(f"\nReporte de clasificación:")
print(classification_report(y_test, y_pred, target_names=target_names))

# =============================================================================
# TABLA RESUMEN Y JUSTIFICACIÓN
# =============================================================================

print("\n" + "=" * 80)
print("TABLA RESUMEN DE MEJORES RESULTADOS POR COMBINACIÓN")
print("=" * 80)

# Top 10 mejores resultados
top_results = results_df.nlargest(10, 'mean_test_f1')[['representation', 'model', 'balancing', 'mean_test_f1', 'std_test_f1']]
print(top_results.round(4))

print("\n" + "=" * 80)
print("JUSTIFICACIÓN DE LA ELECCIÓN DEL MODELO FINAL")
print("=" * 80)

print(f"Se seleccionó la combinación:")
print(f"• Representación: {best_result['representation']}")
print(f"• Modelo: {best_result['model']}")
print(f"• Estrategia de balanceo: {best_result['balancing']}")
print(f"• F1-macro en validación cruzada: {best_result['mean_test_f1']:.4f}")

justification = {
    'CountBinary': "Vectorización binaria simple que captura presencia/ausencia de términos.",
    'CountFreq': "Vectorización por frecuencia que considera repeticiones de términos.",
    'TFIDF': "TF-IDF pondera términos por importancia en el corpus.",
    'TFIDF_ngrams': "TF-IDF con n-gramas captura frases y contextos locales.",
    'LogisticRegression': "Modelo lineal interpretable y eficiente para clasificación.",
    'RandomForest': "Ensemble robusto que maneja bien relaciones no lineales.",
    'SVM': "Buen rendimiento en espacios de alta dimensionalidad.",
    'NaiveBayes': "Modelo probabilístico rápido y eficiente para texto.",
    'none': "Sin balanceo - datos originales.",
    'oversample': "Oversampling de clases minoritarias.",
    'undersample': "Undersampling de clases mayoritarias.",
    'smote': "SMOTE genera muestras sintéticas de clases minoritarias."
}

print(f"\nJustificación técnica:")
print(f"- {justification[best_result['representation']]}")
print(f"- {justification[best_result['model']]}")
print(f"- {justification[best_result['balancing']]}")

# =============================================================================
# VERIFICACIÓN DEL OBJETIVO Y GUARDADO
# =============================================================================

if test_f1_macro >= 0.70:
    print(f"\n✅ ¡OBJETIVO CUMPLIDO! F1-macro = {test_f1_macro:.4f} >= 0.70")
    
    # Guardar modelo final
    model_info = {
        'model': final_model,
        'vectorizer': best_vectorizer,
        'representation': best_result['representation'],
        'balancing': best_result['balancing'],
        'test_metrics': {
            'f1_macro': test_f1_macro,
            'f1_weighted': test_f1_weighted,
            'accuracy': test_accuracy
        },
        'feature_names': best_vectorizer.get_feature_names_out().tolist() + (sel_cols if lexicon_sel else [])
    }
    
    model_filename = f'best_model_{best_result["model"]}_{best_result["representation"]}_{best_result["balancing"]}_f1_{test_f1_macro:.4f}.pkl'
    with open(model_filename, 'wb') as f:
        pickle.dump(model_info, f)
    print(f"Modelo guardado como: {model_filename}")
    
else:
    print(f"\n❌ OBJETIVO NO CUMPLIDO: F1-macro = {test_f1_macro:.4f} < 0.70")
    print("Recomendaciones:")
    print("1. Probar técnicas más avanzadas de balanceo")
    print("2. Aumentar características léxicas y de sentimiento")
    print("3. Probar modelos ensemble más complejos")
    print("4. Mejorar el preprocesamiento de texto")

print("\n" + "=" * 80)
print("EXPERIMENTACIÓN COMPLETADA")
print("=" * 80)

Distribución de clases en el dataset completo:
target
1      547
2      730
3     2121
4     5878
5    20936
Name: count, dtype: int64

Tamaño train: 24169
Tamaño test: 6043
Distribución en train: [    0   438   584  1697  4702 16748]
Distribución en test: [   0  109  146  424 1176 4188]
COMIENZO DE EXPERIMENTACIÓN CON 5-FOLD CV

--- Probando representación: CountBinary ---
  Modelo: LogisticRegression | Balanceo: none
    Error: 'test_f1_macro'
  Modelo: LogisticRegression | Balanceo: oversample
    Error: 'test_f1_macro'
  Modelo: LogisticRegression | Balanceo: undersample
    Error: 'test_f1_macro'
  Modelo: LogisticRegression | Balanceo: smote
    Error: 'test_f1_macro'
  Modelo: RandomForest | Balanceo: none
    Error: 'test_f1_macro'
  Modelo: RandomForest | Balanceo: oversample
    Error: 'test_f1_macro'
  Modelo: RandomForest | Balanceo: undersample
    Error: 'test_f1_macro'
  Modelo: RandomForest | Balanceo: smote
    Error: 'test_f1_macro'
  Modelo: SVM | Balanceo: none
